<a href="https://colab.research.google.com/github/Yohk2291/MachineLearning/blob/master/ML/Titanic_baseline190428_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Titanic

2019年4月28日作成モデルをColaboratoryで再アップ

In [0]:
!git clone https://github.com/Yohk2291/MachineLearning.git
%cd MachineLearning/
%cd ML/

Cloning into 'MachineLearning'...
remote: Enumerating objects: 100, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 100 (delta 36), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (100/100), 1.98 MiB | 6.34 MiB/s, done.
Resolving deltas: 100% (36/36), done.
/content/MachineLearning/ML/MachineLearning/ML/MachineLearning
/content/MachineLearning/ML/MachineLearning/ML/MachineLearning/ML


## 各種ライブラリ、パッケージのインポート

In [0]:
import numpy as np
import pandas as pd
from IPython.display import display
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


## csvの読み込み

In [0]:
# データ型を指定して読み込み
my_dtype = {'Name':object,
            'Sex':object,
            'Embarked':object}


df = pd.read_csv('./data/train.csv', header=0, dtype=my_dtype)
df_s = pd.read_csv('./data/test.csv', header=0, dtype=my_dtype)
sample_submission = pd.read_csv('./data/gender_submission.csv')
display(df.head())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [0]:
X = df.iloc[:,2:] # PassengerId, Survivedを除外(ID, y)
X = X.drop(['Ticket','Cabin'], axis=1) # Ticket, Cabinのデータは今回使用しない
ID = df.iloc[:,[0]]
y = df.iloc[:,1]


display(X.head())
print(X.dtypes)

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S


Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Embarked     object
dtype: object


In [0]:
#表示列数変更
pd.options.display.max_columns = 50

## Trainデータの前処理

In [0]:
#名前の敬称抽出
X['Name'] = X.Name.str.extract('([A-Za-z]+)\.', expand=False)
X.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,Mr,male,22.0,1,0,7.2500,S
1,1,Mrs,female,38.0,1,0,71.2833,C
2,3,Miss,female,26.0,0,0,7.9250,S
3,1,Mrs,female,35.0,1,0,53.1000,S
4,3,Mr,male,35.0,0,0,8.0500,S


In [0]:
#カテゴリ変数（'Name','Sex','Embarked'）をone-hot-encoding（Nanも１カテゴリとする）
ohe_columns = ['Name','Sex','Embarked']
X_ohe = pd.get_dummies(X,dummy_na=True,columns=ohe_columns)

print('X_ohe.shape:(%i,%i)'%X_ohe.shape)
display(X_ohe.head())

X_ohe.shape:(891,30)


,Pclass,Age,SibSp,Parch,Fare,Name_Capt,Name_Col,Name_Countess,Name_Don,Name_Dr,Name_Jonkheer,Name_Lady,Name_Major,Name_Master,Name_Miss,Name_Mlle,Name_Mme,Name_Mr,Name_Mrs,Name_Ms,Name_Rev,Name_Sir,Name_nan,Sex_female,Sex_male,Sex_nan,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
0,3,22.0,1,0,7.2500,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0
1,1,38.0,1,0,71.2833,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0
2,3,26.0,0,0,7.9250,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
3,1,35.0,1,0,53.1000,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0
4,3,35.0,0,0,8.0500,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0


In [0]:
#欠損値Nanを平均値で補完
imp = SimpleImputer()
imp.fit(X_ohe)

X_ohe_columns = X_ohe.columns.values
X_ohe = pd.DataFrame(imp.transform(X_ohe), columns=X_ohe_columns)

display(X_ohe.head())

,Pclass,Age,SibSp,Parch,Fare,Name_Capt,Name_Col,Name_Countess,Name_Don,Name_Dr,Name_Jonkheer,Name_Lady,Name_Major,Name_Master,Name_Miss,Name_Mlle,Name_Mme,Name_Mr,Name_Mrs,Name_Ms,Name_Rev,Name_Sir,Name_nan,Sex_female,Sex_male,Sex_nan,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
0,3.0,22.0,1.0,0.0,7.2500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,1.0,38.0,1.0,0.0,71.2833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,3.0,26.0,0.0,0.0,7.9250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.0,35.0,1.0,0.0,53.1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,3.0,35.0,0.0,0.0,8.0500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [0]:
selector = RFE(RandomForestClassifier(n_estimators=100,random_state=1),n_features_to_select=10,step=.05)

selector.fit(X_ohe,y)

X_fin = pd.DataFrame(selector.transform(X_ohe),columns=X_ohe_columns[selector.support_])

print('X_fin shape:(%i,%i)'%X_fin.shape)
display(X_fin.head())

X_fin shape:(891,10)


,Pclass,Age,SibSp,Parch,Fare,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male
0,3.0,22.0,1.0,0.0,7.2500,0.0,1.0,0.0,0.0,1.0
1,1.0,38.0,1.0,0.0,71.2833,0.0,0.0,1.0,1.0,0.0
2,3.0,26.0,0.0,0.0,7.9250,1.0,0.0,0.0,1.0,0.0
3,1.0,35.0,1.0,0.0,53.1000,0.0,0.0,1.0,1.0,0.0
4,3.0,35.0,0.0,0.0,8.0500,0.0,1.0,0.0,0.0,1.0


## Testデータの前処理

In [0]:
ID_s = df_s.iloc[:,[0]]
X_s = df_s.iloc[:,1:]
X_s = X_s.drop(['Ticket','Cabin'], axis=1)

print('Raw Shape:(%i,%i)'%df_s.shape)
print('X_shape:(%i,%i)'%X_s.shape)
print('-------------------------------')
print(X_s.dtypes)

Raw Shape:(418,11)
X_shape:(418,8)
-------------------------------
Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Embarked     object
dtype: object


In [0]:
#名前の敬称抽出
X_s['Name'] = X_s.Name.str.extract('([A-Za-z]+)\.', expand=False)
X_s.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,Mr,male,34.5,0,0,7.8292,Q
1,3,Mrs,female,47.0,1,0,7.0000,S
2,2,Mr,male,62.0,0,0,9.6875,Q
3,3,Mr,male,27.0,0,0,8.6625,S
4,3,Mrs,female,22.0,1,1,12.2875,S


In [0]:
X_ohe_s = pd.get_dummies(X_s,dummy_na=True,columns=ohe_columns)
print('X_ohe_s shape:(%i,%i)'%X_ohe_s.shape)
display(X_ohe_s.head())

X_ohe_s shape:(418,22)


,Pclass,Age,SibSp,Parch,Fare,Name_Col,Name_Dona,Name_Dr,Name_Master,Name_Miss,Name_Mr,Name_Mrs,Name_Ms,Name_Rev,Name_nan,Sex_female,Sex_male,Sex_nan,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
0,3,34.5,0,0,7.8292,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0
1,3,47.0,1,0,7.0000,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0
2,2,62.0,0,0,9.6875,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0
3,3,27.0,0,0,8.6625,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0
4,3,22.0,1,1,12.2875,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0


In [0]:
cols_model = set(X_ohe.columns.values)
cols_score = set(X_ohe_s.columns.values)

diff1 = cols_model - cols_score
print('Modelのみ:%s'%diff1)

diff2 = cols_score - cols_model
print('Scoreのみ:%s'%diff2)

Modelのみ:{'Name_Mlle', 'Name_Capt', 'Name_Sir', 'Name_Major', 'Name_Jonkheer', 'Name_Mme', 'Name_Don', 'Name_Lady', 'Name_Countess'}
Scoreのみ:{'Name_Dona'}


In [0]:
df_cols_m = pd.DataFrame(None,
                         columns=X_ohe_columns,
                         dtype=float)
display(df_cols_m)

,Pclass,Age,SibSp,Parch,Fare,Name_Capt,Name_Col,Name_Countess,Name_Don,Name_Dr,Name_Jonkheer,Name_Lady,Name_Major,Name_Master,Name_Miss,Name_Mlle,Name_Mme,Name_Mr,Name_Mrs,Name_Ms,Name_Rev,Name_Sir,Name_nan,Sex_female,Sex_male,Sex_nan,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan


In [0]:
X_ohe_s2 = pd.concat([df_cols_m, X_ohe_s])
print(X_ohe_s2.shape)
display(X_ohe_s2.head(3))

(418, 31)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,Age,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan,Fare,Name_Capt,Name_Col,Name_Countess,Name_Don,Name_Dona,Name_Dr,Name_Jonkheer,Name_Lady,Name_Major,Name_Master,Name_Miss,Name_Mlle,Name_Mme,Name_Mr,Name_Mrs,Name_Ms,Name_Rev,Name_Sir,Name_nan,Parch,Pclass,Sex_female,Sex_male,Sex_nan,SibSp
0,34.5,0.0,1.0,0.0,0.0,7.8292,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,1.0,0.0,0.0,0.0,NaN,0.0,0.0,3.0,0.0,1.0,0.0,0.0
1,47.0,0.0,0.0,1.0,0.0,7.0000,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,1.0,0.0,0.0,NaN,0.0,0.0,3.0,1.0,0.0,0.0,1.0
2,62.0,0.0,1.0,0.0,0.0,9.6875,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,1.0,0.0,0.0,0.0,NaN,0.0,0.0,2.0,0.0,1.0,0.0,0.0


In [0]:
set_Xm = set(X_ohe.columns.values)
set_Xs = set(X_ohe_s.columns.values)

X_ohe_s3 = X_ohe_s2.drop(list(set_Xs-set_Xm),axis=1)

print(X_ohe_s3.shape)
display(X_ohe_s3.head(3))

(418, 30)


,Age,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan,Fare,Name_Capt,Name_Col,Name_Countess,Name_Don,Name_Dr,Name_Jonkheer,Name_Lady,Name_Major,Name_Master,Name_Miss,Name_Mlle,Name_Mme,Name_Mr,Name_Mrs,Name_Ms,Name_Rev,Name_Sir,Name_nan,Parch,Pclass,Sex_female,Sex_male,Sex_nan,SibSp
0,34.5,0.0,1.0,0.0,0.0,7.8292,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,1.0,0.0,0.0,0.0,NaN,0.0,0.0,3.0,0.0,1.0,0.0,0.0
1,47.0,0.0,0.0,1.0,0.0,7.0000,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,1.0,0.0,0.0,NaN,0.0,0.0,3.0,1.0,0.0,0.0,1.0
2,62.0,0.0,1.0,0.0,0.0,9.6875,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,1.0,0.0,0.0,0.0,NaN,0.0,0.0,2.0,0.0,1.0,0.0,0.0


In [0]:
X_ohe_s3.loc[:,list(set_Xm-set_Xs)] = X_ohe_s3.loc[:,list(set_Xm-set_Xs)].fillna(0,axis=1)
X_ohe_s3.head(3)

,Age,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan,Fare,Name_Capt,Name_Col,Name_Countess,Name_Don,Name_Dr,Name_Jonkheer,Name_Lady,Name_Major,Name_Master,Name_Miss,Name_Mlle,Name_Mme,Name_Mr,Name_Mrs,Name_Ms,Name_Rev,Name_Sir,Name_nan,Parch,Pclass,Sex_female,Sex_male,Sex_nan,SibSp
0,34.5,0.0,1.0,0.0,0.0,7.8292,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0
1,47.0,0.0,0.0,1.0,0.0,7.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,1.0
2,62.0,0.0,1.0,0.0,0.0,9.6875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0


In [0]:
X_ohe_s3 = X_ohe_s3.reindex(X_ohe.columns.values,axis=1)
X_ohe_s3.head(3)

,Pclass,Age,SibSp,Parch,Fare,Name_Capt,Name_Col,Name_Countess,Name_Don,Name_Dr,Name_Jonkheer,Name_Lady,Name_Major,Name_Master,Name_Miss,Name_Mlle,Name_Mme,Name_Mr,Name_Mrs,Name_Ms,Name_Rev,Name_Sir,Name_nan,Sex_female,Sex_male,Sex_nan,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
0,3.0,34.5,0.0,0.0,7.8292,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,3.0,47.0,1.0,0.0,7.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2.0,62.0,0.0,0.0,9.6875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [0]:
print('欠損個数（数値変数の欠損補完前）',X_ohe_s3.isnull().sum().sum())
X_ohe_s4 = pd.DataFrame(imp.transform(X_ohe_s3),columns=X_ohe_columns)
print('欠損個数（数値変数の欠損補完後）',X_ohe_s4.isnull().sum().sum())

欠損個数（数値変数の欠損補完前） 87
欠損個数（数値変数の欠損補完後） 0


In [0]:
X_fin_s = X_ohe_s4.loc[:, X_ohe_columns[selector.support_]]
print(X_fin_s.shape)
X_fin_s.head(20)

(418, 10)


,Pclass,Age,SibSp,Parch,Fare,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male
0,3.0,34.500000,0.0,0.0,7.8292,0.0,1.0,0.0,0.0,1.0
1,3.0,47.000000,1.0,0.0,7.0000,0.0,0.0,1.0,1.0,0.0
2,2.0,62.000000,0.0,0.0,9.6875,0.0,1.0,0.0,0.0,1.0
3,3.0,27.000000,0.0,0.0,8.6625,0.0,1.0,0.0,0.0,1.0
4,3.0,22.000000,1.0,1.0,12.2875,0.0,0.0,1.0,1.0,0.0
5,3.0,14.000000,0.0,0.0,9.2250,0.0,1.0,0.0,0.0,1.0
6,3.0,30.000000,0.0,0.0,7.6292,1.0,0.0,0.0,1.0,0.0
7,2.0,26.000000,1.0,1.0,29.0000,0.0,1.0,0.0,0.0,1.0
8,3.0,18.000000,0.0,0.0,7.2292,0.0,0.0,1.0,1.0,0.0
9,3.0,21.000000,2.0,0.0,24.1500,0.0,1.0,0.0,0.0,1.0


## Trainデータから、訓練用と検証用に分割

In [0]:
# holdout
X_train,X_valid,y_train,y_valid=train_test_split(X_fin,y,test_size=0.15,random_state=1)

In [0]:
# set pipelines for different algorithms
pipelines = {
    'knn':
        Pipeline([('scl',StandardScaler()),
                  ('est',KNeighborsClassifier())]),
    'logistic':
        Pipeline([('scl',StandardScaler()),
                  ('est',LogisticRegression(random_state=1))]),
    'rsvc':
        Pipeline([('scl',StandardScaler()),
                  ('est',SVC(C=1.0, kernel='rbf', class_weight='balanced', random_state=1))]),
    'lsvc':
        Pipeline([('scl',StandardScaler()),
                  ('est',LinearSVC(C=1.0, class_weight='balanced', random_state=1))]),
    'tree':
        Pipeline([('scl',StandardScaler()),
                  ('est',DecisionTreeClassifier(random_state=1))]),
    'rf':
        Pipeline([('scl',StandardScaler()),
                  ('est',RandomForestClassifier(random_state=1))]),
    'gb':
        Pipeline([('scl',StandardScaler()),
                  ('est',GradientBoostingClassifier(random_state=1))]),
    'mlp':
        Pipeline([('scl',StandardScaler()),
                  ('est',MLPClassifier(hidden_layer_sizes=(3,3),max_iter=1000))]),
    'lgb':
        Pipeline([('scl', StandardScaler()), ('est', LGBMClassifier(random_state=1))])
}
# fit & evaluation
scores = {}
for pipe_name, pipeline in pipelines.items():
    pipeline.fit(X_train, y_train)
    scores[(pipe_name,'train')] = accuracy_score(y_train, pipeline.predict(X_train))
    scores[(pipe_name,'valid')] = accuracy_score(y_valid, pipeline.predict(X_valid))

pd.Series(scores).unstack()

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


,train,valid
gb,0.915456,0.828358
knn,0.859974,0.791045
lgb,0.945839,0.768657
logistic,0.821664,0.783582
lsvc,0.813738,0.791045
mlp,0.840159,0.768657
rf,0.969617,0.791045
rsvc,0.844122,0.798507
tree,0.985469,0.701493


In [0]:
logistic = pipelines['logistic']

logistic.fit(X_train, y_train)
oof = logistic.predict(X_valid)
print('score',round(accuracy_score(y_valid, oof)*100, 2))

test_pred = logistic.predict(X_fin_s)

sample_submission['Survived'] = test_pred
sample_submission.to_csv('train_test_split.csv',index=False)

score 78.36


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
